### **Estruturas Criptográficas - 2022-2023**
### **Grupo 7**
#### **TP1. Problema 1**

Pretende-se a criação de uma comunicação privada e assíncrona entre um agente *emitter* (Alice) e um agente *receiver* (Bob).

A comunicação começa com a troca das duas chaves públicas de cada agente (criadas para construir tanto a chave para cifra como a chave para a autenticação). Depois, cada agente irá gerar as duas respetivas chaves partilhadas, *cipher_key* (chave de cifra) e *mac_key*​ (chave de autenticação).

As assinaturas digitais (neste caso **ECDSA**) serão usadas para garantir a autenticação dos agentes.

Note-se que a Alice envia mensagens ao Bob que sejam autenticadas com a chave partilhada de autenticação e cifradas com a chave de cifra.

**Resolução**:

In [85]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
import os
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from pickle import dumps, loads


Como já referido, numa primeira fase, cada agente deverá gerar os pares de chaves assimétricas de modo a poderem acordar num par de chaves partilhadas. Para tal, utilizou-se o protocolo (seguro) **ECDH** (Elliptic Curve Diffie-Hellman Key Exchange).

Como também já referido, é necessário que cada agente gere dois pares de chaves assimétricas, uma para cifrar as mensagems e a outra para a autenticação dos agentes. As respetivas chaves públicas serão então trocadas pelos dois agentes, por um canal controlado pelo atacante. No entanto, o atacante apenas conhecerá as chaves públicas, e por isso não consegue gerar as chaves partilhadas, pois não tem a informação das chaves privadas.

A função *generate_keys* trata de gerar as 4 chaves referidas.

In [86]:
def generate_keys():
    cipher_sk = ec.generate_private_key(ec.SECP384R1())
    cipher_pk = cipher_sk.public_key()

    mac_sk = ec.generate_private_key(ec.SECP384R1())
    mac_pk = mac_sk.public_key()

    return [cipher_sk, cipher_pk, mac_sk, mac_pk]

A função *generate_shared_keys* trata de gerar as chaves partilhadas, *cipher_key* e *mac_key*.

É de notar que çara a maioria das aplicações, a chave partilhada deve ser passada a uma função de derivação de chaves (**KDF**). Isso permite a mistura de informações adicionais na chave, a derivação de várias chaves e a destruição de qualquer estrutura que possa estar presente.

In [87]:
def generate_shared_keys(cipher_sk, received_cipher_pk, mac_sk, received_mac_pk):
    cipher_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'handshake data',
    ).derive(cipher_sk.exchange(ec.ECDH(), received_cipher_pk))

    mac_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'handshake data',
    ).derive(mac_sk.exchange(ec.ECDH(), received_mac_pk))

    return cipher_key, mac_key

Relativamente à autenticação dos agentes (não apenas para garantir a autenticidade, mas também a integridade e o não-repúdio na troca de chaves), foi utilizado o algoritmo de assinatura **ECDSA**. A chave privada é usada para assinar a mensagem e a chave pública para verificar a validade da assinatura. Assim, um agente consegue verificar se a mensagem que recebeu é fidedigna.

A função *sign_message* é usada para assinar uma mensagem, retornando a assinatura, a própria mensagem e a chave pública.
Note-se que para verificar é necessário usar o mesmo algoritmo de *hashing*.

In [88]:
def sign_message(message):
    private_key = ec.generate_private_key(
        ec.SECP384R1()
    )
    signature = private_key.sign(
        message,
        ec.ECDSA(hashes.SHA256())
    )
    packet = {'signature' : signature, 
              'message'   : message, 
              'ecdsa_pk'  : private_key.public_key().public_bytes(
                                                        encoding=serialization.Encoding.PEM,
                                                        format=serialization.PublicFormat.SubjectPublicKeyInfo
                                                     )
             }
    return packet

A função *encrypt* será usada pela Alice para cifrar a mensagem a enviar ao Bob. A cifra a ser utilizada é a cifra simétrica **AES** no modo **GCM**, com a chave de cifra partilhada. Foi escolhida esta cifra por fornecer tanto confidencialidade como integridade do texto cifrado e por fornecer integridade para dados associados que não são cifrados. Além disso, o modo **GCM** é seguro contra ataques aos *nonce*. Neste tipo de ataque, o atacante envia a mensagem copiada para o destinatário diversas vezes. O *nonce*, que é gerado a partir de uma função de hash em modo XOF (SHAKE256), permite identificar cada mensagem com um identificador exclusivo, o que reduz significativamente a probabilidade desses ataques serem bem sucedidos.

In [89]:
def encrypt(plaintext, cipher_key, mac_key, ad):
    
    digest = hashes.Hash(hashes.SHAKE256(16))
    nonce = digest.finalize()

    aesgcm = AESGCM(cipher_key)
    ciphertext = aesgcm.encrypt(nonce, plaintext, ad)

    h = hmac.HMAC(mac_key, hashes.SHA256())
    h.update(ciphertext)
    h.update(ad)
    tag = h.finalize()

    return (ciphertext, nonce, tag)

def decrypt(ciphertext, nonce, tag, cipher_key, mac_key, ad):
    h = hmac.HMAC(mac_key, hashes.SHA256())
    h.update(ciphertext)
    h.update(ad)

    try:
        h.verify(tag)
    except:
        return 'ERROR --- Different MAC key used'
    else:
        # decrypt ciphertext
        aesgcm = AESGCM(cipher_key)
        original_plaintext = aesgcm.decrypt(nonce, ciphertext, ad)
        return original_plaintext

##### **Comunicação**

In [90]:
async def alice(queue):
    keys = generate_keys()
    cipher_sk = keys[0]
    cipher_pk = keys[1]
    mac_sk = keys[2]
    mac_pk = keys[3]

    public_keys = {'cipher_pk' : cipher_pk.public_bytes(
                                    encoding=serialization.Encoding.PEM,
                                    format=serialization.PublicFormat.SubjectPublicKeyInfo
                                 ),
                   'mac_pk'    : mac_pk.public_bytes(
                                    encoding=serialization.Encoding.PEM,
                                    format=serialization.PublicFormat.SubjectPublicKeyInfo
                                 )
                  }

    print("Alice is sending her public keys (signed by her).")
    await queue.put(sign_message(public_keys))

    bob_pks = await queue.get()
    print("Alice is receiving Bob's public keys.")

    print("Alice is verifying Bob's message signature.")
    bob_ecdsa_pk = serialization.load_pem_public_key(bob_pks['ecdsa_pk'])

    try:
        bob_ecdsa_pk.verify(bob_pks['signature'], bob_pks['message'], ec.ECDSA(hashes.SHA256()))
    except:
        return 'ERROR --- Different ECDSA key used'
    else:
        print("Bob's message is authentic!")

        received_cipher_pk = serialization.load_pem_public_key(bob_pks['message']['cipher_pk'])
        received_mac_pk = serialization.load_pem_public_key(bob_pks['message']['mac_pk'])

        # generate shared keys
        cipher_key, mac_key = generate_shared_keys(cipher_sk, received_cipher_pk, mac_sk, received_mac_pk)

        msg_to_send = '(FROM ALICE) Hello Bob!'
        print('ORIGINAL: ' + msg_to_send)

        # cipher message
        print("Encrypting Alice's message.")
        ad = os.urandom(16)
        ciphertext, nonce, tag = encrypt(msg_to_send, cipher_key, mac_key, ad)

        # send packet
        packet = {
            'ciphertext' : ciphertext,
            'nonce' : nonce,
            'tag' : tag,
            'ad' : ad
        }
        print("Sending Alice's message.")

In [91]:
async def bob(queue):
    keys = generate_keys()
    cipher_sk = keys[0]
    cipher_pk = keys[1]
    mac_sk = keys[2]
    mac_pk = keys[3]

    public_keys = {'cipher_pk': cipher_pk.public_bytes(
                                    encoding=serialization.Encoding.PEM,
                                    format=serialization.PublicFormat.SubjectPublicKeyInfo
                                ),
                      'mac_pk': mac_pk.public_bytes(
                                    encoding=serialization.Encoding.PEM,
                                    format=serialization.PublicFormat.SubjectPublicKeyInfo
                                )
                  }
    
    alice_pks = await queue.get()
    print("Bob is receiving Alice's public keys.")

    alice_ecdsa_pk = serialization.load_pem_public_key(alice_pks['ecdsa_pk'])

    try:
        alice_ecdsa_pk.verify(alice_pks['signature'], alice_pks['message'], ec.ECDSA(hashes.SHA256()))
    except:
        return 'ERROR --- Different ECDSA key used'
    else:
        print("Alice's message is authentic!")

        received_cipher_pk = serialization.load_pem_public_key(alice_pks['message']['cipher_pk'])
        received_mac_pk = serialization.load_pem_public_key(alice_pks['message']['mac_pk'])

        # generate shared keys
        cipher_key, mac_key = generate_shared_keys(cipher_sk, received_cipher_pk, mac_sk, received_mac_pk)

        print("Bob is sending his public keys (signed by him).")
        await queue.put(sign_message(public_keys))

        # receives ciphertext
        packet = await queue.get()
        print('Bob received ciphertext and tries to decrypt it.')

        plaintext = decrypt(packet['ciphertext'], packet['nonce'], packet['tag'], cipher_key, mac_key, packet['ad'])

        return plaintext

In [ ]:
loop = asyncio.get_event_loop()
queue = asyncio.Queue()
asyncio.ensure_future(alice(queue))
loop.run_until_complete(bob(queue))